In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.model_selection import KFold
import sklearn
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import pandas as pd

In [2]:
f = pd.read_csv('/is/prices/tg_join_lab.csv', sep='|')

print(f.shape)
b = pd.read_csv('/is/prices/tg_bh.csv', sep='|')
f = f.merge(b, how='inner', on='ticker')
f['obs'] = 1
f = f.sort_values('date', ascending=False)
print(f.shape)

(93490, 300)
(93490, 302)


In [11]:
j_neut = 0.2
j_pos = 2.0
j_neg = 2.0
i = 1
label_neut = f'label_{i}_{j_neut}'
label_pos = f'label_{i}_{j_pos}'
label_neg = f'label_{i}_{j_neg}'
print('neut: ', f[label_neut][f[label_neut]==0].shape)
print('neg: ', f[label_neg][f[label_neg]==-1].shape)
print('pos: ', f[label_pos][f[label_pos]==1].shape)

neut:  (11208,)
neg:  (11627,)
pos:  (12971,)


In [4]:
9*2*4*4*62

17856

#### Финальный репорт

In [5]:
from sklearn import metrics
field = 'text_reg_spell_stop_lem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 21, 28, 36, 45, 55, 66] 
max_dfs = [0.5]
ngram_range_maxs = [1]
j_neuts = [0.2, 0.5]
j_poss = [2.0, 3.0, 4.0, 6.0] 
j_negs = [2.0, 3.0, 4.0, 6.0]
days = [1]


counter = 0
i = 1
t = 4

# split
for ticker in  f.ticker.unique(): #['sber']: #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                for day in days:
                    try:

                        ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                        ps = ps.loc[:,~ps.columns.duplicated()]
                        if t == 0:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                           ]).sample(frac=1, random_state=44)

                        else:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neut]==0)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                            ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                            ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                           ]).sample(frac=1, random_state=44)


                        x_train = df[field]
                        y_train = df['sentiment'] 

                    # fit
                        for ngram_range_max in ngram_range_maxs:
                            for min_df in min_dfs:
                                for max_df in max_dfs:

                                    tf_idf = TfidfVectorizer(min_df=min_df, \
                                                             max_df=max_df, \
                                                             ngram_range=(1, ngram_range_max))

                                    train_features = tf_idf.fit_transform(x_train)


                                    model = MultinomialNB(alpha=0.1)
                                    model.fit(train_features, y_train)
                                    y_train_pred = model.predict(train_features)
    # В идеале волатильность взять за эти же лаги наверное
                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                           [['text_reg_spell_stop_lem', 'obs', 'date',\
                                                             f'change_close_{day}', 'bh', 'close']]).\
        reset_index().drop('index', axis=1)

                                    y_pred = model.predict(tf_idf.transform(df_temp.text_reg_spell_stop_lem))

                                    df_temp['sentiment'] = y_pred

                                    df_temp['neg'] = 0
                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                    df_temp['pos'] = 0
                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                    df_temp['neut'] = 0
                                    df_temp['neut'][df_temp.sentiment==0] = 1

                                    df_temp = df_temp.drop_duplicates().\
                                    groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

    # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
    # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                    df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                    #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                    df_temp = df_temp.sort_values('date', ascending=True)

                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                    df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                    # просто сумма
                                    df_temp['sum_sentiment'] = 0
                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

    # дивергенция c добавлением 
                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                              / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                    #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                    res = dict()
                                    res['counter'] = [counter]
                                    res['ticker'] = [ticker]
                                    res['date'] = [day]

                                    res['min_df'] = [min_df]

                                    res['j_neut'] = [j_neut]
                                    res['j_pos'] = [j_pos] 
                                    res['j_neg'] = [j_neg] 

                                    res['train_size'] = [y_train.shape[0]]
                                    res['test_size'] = [y_pred.shape[0]]

                                    res['obs'] = \
                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                    res['bh'] = df_temp.bh.max()
                                    res['original_return_isna_all'] = \
                                    [(df_temp[f'change_close_{day}']\
                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                    res['model_return_isna_all'] = \
                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                    res['model_return_isna_short'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_return_isna_long'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                    res['model_diver_return_isna_all'] = \
                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                    res['model_diver_return_isna_short'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_diver_return_isna_long'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='macro')]


                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='micro')]


                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='micro')]


                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='macro')]

                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='micro')]


                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='macro')]



                                    x = df_temp[['sum_sentiment', 'close_lag_1']]
                                    x.sum_sentiment = x.sum_sentiment.astype(str)

                                    x = pd.get_dummies(x, drop_first=True)
                                    y = df_temp.close
                                    x2 = sm.add_constant(x)
                                    lm = sm.OLS(y, x2)
                                    res_lm = lm.fit()

                                    res['const'] = res_lm.params[0]
                                    res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                    res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                    res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                    res['const_p'] = res_lm.pvalues[0]
                                    res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                    res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                    res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                    res['r2'] = res_lm.rsquared
                                    res['nobs'] = res_lm.nobs
                                    counter += 1
                                    print(counter, end='\r')


                                    res = pd.DataFrame.from_dict(res)
                                    
                                    out = out.append(pd.DataFrame(res))
                                    if counter % 100 == 0: 
                                        out.to_excel(f'/is/res/nb/nb_model_ind_final.xlsx', index=False)
                                    
                    except:
                        continue
    

                    


out.to_excel(f'/is/res/nb/nb_model_ind_final.xlsx', index=False)
out

vtbr
gmkn
zill
alrs
uwgn
tcsg
belu
lnta
hydr
sber
irao
afks
gche
amez
sngs
nmtp
moex
smlt
yndx
krot
five
chgz
gazp
mvid
plzl
pikk
nknc
upro
paza
rosn
zvez
ucss
magn
mtlr
iskj
rtkm0
qiwi8
tatn6
lkoh4
rasp2
aflt0
enru8
bane6
rbcm4
phor2
nlmk0
rual8
mgnt6
hals4
aptk2
lsng0
kmaz8
mtss6
nvtk4
chmf2
rsti0
dsky8
mail6
tanl4
lsrg2
rlmn0
trmk8


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,vtbr,1,5,0.2,2.0,2.0,30909,6264,6243,...,0.000535,0.984228,0.000123,0.000078,0.003992,0.000000e+00,0.044127,0.219362,0.982910,852.0
0,1,vtbr,1,10,0.2,2.0,2.0,30909,6264,6243,...,0.000574,0.984111,0.000062,0.000046,0.001949,0.000000e+00,0.303562,0.469253,0.982850,852.0
0,2,vtbr,1,15,0.2,2.0,2.0,30909,6264,6243,...,0.000582,0.983928,0.000054,0.000050,0.001537,0.000000e+00,0.369043,0.425925,0.982847,852.0
0,3,vtbr,1,21,0.2,2.0,2.0,30909,6264,6243,...,0.000581,0.984116,0.000068,0.000013,0.001734,0.000000e+00,0.264288,0.840128,0.982857,852.0
0,4,vtbr,1,28,0.2,2.0,2.0,30909,6264,6243,...,0.000619,0.984032,-0.000021,0.000006,0.000858,0.000000e+00,0.727365,0.921994,0.982832,852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,17851,trmk,1,28,0.5,6.0,6.0,4914,180,179,...,15.769988,0.729318,-0.337080,-1.561980,0.000020,1.512917e-19,0.709185,0.102991,0.645399,85.0
0,17852,trmk,1,36,0.5,6.0,6.0,4914,180,179,...,15.232391,0.729456,-0.222615,-0.016312,0.000047,3.179073e-19,0.811583,0.985939,0.632298,85.0
0,17853,trmk,1,45,0.5,6.0,6.0,4914,180,179,...,15.372358,0.726561,-0.769265,0.604388,0.000035,1.733385e-19,0.399574,0.508043,0.641953,85.0
0,17854,trmk,1,55,0.5,6.0,6.0,4914,180,179,...,16.051362,0.726135,-1.669117,-0.741723,0.000015,1.285814e-19,0.064713,0.408043,0.647261,85.0


In [6]:
from sklearn import metrics
field = 'text_reg_spell_stop_lem'
out = pd.DataFrame()

min_dfs = [5, 7, 10, 15, 21, 28, 36, 45, 55, 66] 
max_dfs = [0.5]
ngram_range_maxs = [1]
j_neuts = [0.1, 0.2, 0.5, 0.05]
j_poss = [2.0, 3.0, 5.0, 9.0] 
j_negs = [2.0, 3.0, 5.0, 9.0]
days = [1]


counter = 0
i = 1
t = 4

# split
for ticker in  f.ticker.unique(): #['sber']: #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                for day in days:
                    try:

                        ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                        ps = ps.loc[:,~ps.columns.duplicated()]
                        if t == 0:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                           ]).sample(frac=1, random_state=44)

                        else:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neut]==0)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                            ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                            ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                           ]).sample(frac=1, random_state=44)


                        x_train = df[field]
                        y_train = df['sentiment'] 

                    # fit
                        for ngram_range_max in ngram_range_maxs:
                            for min_df in min_dfs:
                                for max_df in max_dfs:

                                    tf_idf = TfidfVectorizer(min_df=min_df, \
                                                             max_df=max_df, \
                                                             ngram_range=(1, ngram_range_max))

                                    train_features = tf_idf.fit_transform(x_train)


                                    model = MultinomialNB(alpha=0.1)
                                    model.fit(train_features, y_train)
                                    y_train_pred = model.predict(train_features)
    # В идеале волатильность взять за эти же лаги наверное
                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                           [['text_reg_spell_stop_lem', 'obs', 'date',\
                                                             f'change_close_{day}', 'bh', 'close']]).\
        reset_index().drop('index', axis=1)

                                    y_pred = model.predict(tf_idf.transform(df_temp.text_reg_spell_stop_lem))

                                    df_temp['sentiment'] = y_pred

                                    df_temp['neg'] = 0
                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                    df_temp['pos'] = 0
                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                    df_temp['neut'] = 0
                                    df_temp['neut'][df_temp.sentiment==0] = 1

                                    df_temp = df_temp.drop_duplicates().\
                                    groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

    # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
    # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                    df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                    #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                    df_temp = df_temp.sort_values('date', ascending=True)

                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                    df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                    # просто сумма
                                    df_temp['sum_sentiment'] = 0
                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

    # дивергенция c добавлением 
                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                              / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                    #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                    res = dict()
                                    res['counter'] = [counter]
                                    res['ticker'] = [ticker]
                                    res['date'] = [day]

                                    res['min_df'] = [min_df]

                                    res['j_neut'] = [j_neut]
                                    res['j_pos'] = [j_pos] 
                                    res['j_neg'] = [j_neg] 

                                    res['train_size'] = [y_train.shape[0]]
                                    res['test_size'] = [y_pred.shape[0]]

                                    res['obs'] = \
                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                    res['bh'] = df_temp.bh.max()
                                    res['original_return_isna_all'] = \
                                    [(df_temp[f'change_close_{day}']\
                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                    res['model_return_isna_all'] = \
                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                    res['model_return_isna_short'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_return_isna_long'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                    res['model_diver_return_isna_all'] = \
                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                    res['model_diver_return_isna_short'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_diver_return_isna_long'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='macro')]


                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='micro')]


                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='micro')]


                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='macro')]

                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='micro')]


                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='macro')]



                                    x = df_temp[['sum_sentiment', 'close_lag_1']]
                                    x.sum_sentiment = x.sum_sentiment.astype(str)

                                    x = pd.get_dummies(x, drop_first=True)
                                    y = df_temp.close
                                    x2 = sm.add_constant(x)
                                    lm = sm.OLS(y, x2)
                                    res_lm = lm.fit()

                                    res['const'] = res_lm.params[0]
                                    res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                    res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                    res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                    res['const_p'] = res_lm.pvalues[0]
                                    res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                    res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                    res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                    res['r2'] = res_lm.rsquared
                                    res['nobs'] = res_lm.nobs
                                    counter += 1
                                    print(counter, end='\r')


                                    res = pd.DataFrame.from_dict(res)
                                    
                                    out = out.append(pd.DataFrame(res))
                                    if counter % 1000 == 0: 
                                        out.to_excel(f'/is/res/nb/nb_model_ind_final2.xlsx', index=False)
                                    
                    except:
                        continue
    

                    


out.to_excel(f'/is/res/nb/nb_model_ind_final2.xlsx', index=False)
out

vtbr
gmkn
zill
alrs
uwgn
tcsg
belu
lnta
hydr
sber
irao
afks
gche
amez
sngs
nmtp
moex9
smlt9
yndx9
krot9
five9
chgz9
gazp9
mvid9
plzl9
pikk9
nknc9
upro9
paza9
rosn9
zvez9
ucss9
magn9
mtlr9
iskj9
rtkm8
qiwi8
tatn8
lkoh8
rasp8
aflt8
enru8
bane8
rbcm8
phor8
nlmk8
rual8
mgnt8
hals8
aptk8
lsng8
kmaz8
mtss8
nvtk8
chmf8
rsti8
dsky8
mail8
tanl8
lsrg8
rlmn8
trmk8


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,vtbr,1,5,0.10,2.0,2.0,16311,6264,6243,...,0.000525,0.984377,0.000141,0.000077,0.004998,0.000000e+00,0.016689,0.238640,0.982943,852.0
0,1,vtbr,1,7,0.10,2.0,2.0,16311,6264,6243,...,0.000525,0.984594,0.000111,0.000091,0.004974,0.000000e+00,0.062237,0.156996,0.982905,852.0
0,2,vtbr,1,10,0.10,2.0,2.0,16311,6264,6243,...,0.000553,0.984134,0.000096,0.000075,0.002805,0.000000e+00,0.109404,0.233291,0.982884,852.0
0,3,vtbr,1,15,0.10,2.0,2.0,16311,6264,6243,...,0.000553,0.984361,0.000073,0.000076,0.002915,0.000000e+00,0.220340,0.224869,0.982869,852.0
0,4,vtbr,1,21,0.10,2.0,2.0,16311,6264,6243,...,0.000589,0.984068,0.000026,0.000052,0.001440,0.000000e+00,0.658136,0.419298,0.982841,852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,39673,trmk,1,28,0.05,9.0,9.0,1977,180,179,...,15.304220,0.727361,0.825639,-0.779243,0.000034,1.282716e-19,0.377293,0.383744,0.645686,85.0
0,39674,trmk,1,36,0.05,9.0,9.0,1977,180,179,...,15.226643,0.725617,0.877826,-0.253445,0.000040,2.765442e-19,0.358748,0.773267,0.638704,85.0
0,39675,trmk,1,45,0.05,9.0,9.0,1977,180,179,...,15.211965,0.734854,-0.079911,-1.220525,0.000039,1.886968e-19,0.931378,0.176901,0.641162,85.0
0,39676,trmk,1,55,0.05,9.0,9.0,1977,180,179,...,15.705568,0.725614,-1.112307,-0.382099,0.000038,1.304992e-18,0.227826,0.708660,0.638589,85.0


In [8]:
out.shape

(39678, 35)